In [2]:
from utils.extract_wound_class import CachedWoundClassifier

classifier = CachedWoundClassifier()
label, probabilities = classifier.predict("careful_this_contain_wound_image/test/Normal/2.jpg")
print(f"Predicted: {label}")

Predicted: Normal


In [2]:
from utils.extract_wound_features import CLIPWoundFeatureExtractor
from utils.extract_wound_class import CachedWoundClassifier

extractor = CLIPWoundFeatureExtractor()
classifier = CachedWoundClassifier()
image_path = 'careful_this_contain_wound_image/Burns/burns (20).jpg'
wound_class, probabilities = classifier.predict(image_path)

features = extractor.extract_features(image_path, wound_class, lang='th')

print(f"\nTop features for wound class: {wound_class}")
for desc, score in features:
    print(f"{desc}: {score:.4f}")


Top features for wound class: Burns
พบบวมพองซึ่งสอดคล้องกับแผลไฟไหม้ระดับที่สอง: 1.0000
ความลึกของแผลไหม้ลามถึงชั้นใต้ผิวหนัง: 0.9414
ผิวหนังไหม้ดำหรือแข็งคล้ายหนัง บ่งชี้แผลไหม้ลึกถึงชั้นผิวหนังทั้งหมด: 0.9111
มีรอยแดงและบวมในบริเวณที่ไหม้: 0.7632
ผิวหนังรอบแผลมีรอยแดงและเจ็บปวด: 0.6871
แผลมีกลิ่นจางๆ ซึ่งพบได้ในแผลไฟไหม้: 0.6744
บริเวณแผลไหม้มีผิวแห้ง แตก และลอกออก: 0.6097
ขอบแผลไหม้ไม่เรียบและมีปฏิกิริยาของเนื้อเยื่อแตกต่างกัน: 0.5775
ผิวหนังลอกออกและเห็นชั้นผิวหนังด้านใน: 0.4127
ไม่มีเลือดออกมาก แต่มีสะเก็ดแผลเริ่มก่อตัว: 0.3827
